In [ ]:
# Install kaggle API
!pip install kaggle

# Upload kaggle.json (API token)
from google.colab import files
files.upload()   # apni kaggle.json file yahan select karo

# Save kaggle.json in correct folder
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d chetankv/dogs-cats-images

In [ ]:
!unzip -q dogs-cats-images.zip -d dogs_cats_dataset


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import os


In [ ]:
import os

# Training aur test directories ka path
train_dir = '/content/dogs_cats_dataset/dataset/training_set'
test_dir = '/content/dogs_cats_dataset/dataset/test_set'

def count_images(directory):
    total = 0
    for root, dirs, files in os.walk(directory):
        total += len([f for f in files if f.endswith(('.jpg', '.png', '.jpeg'))])
    return total

train_count = count_images(train_dir)
test_count = count_images(test_dir)
total_count = train_count + test_count

print(f"Training images: {train_count}")
print(f"Test/Validation images: {test_count}")
print(f"Total images in dataset: {total_count}")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# No augmentation for test/validation
test_datagen = ImageDataGenerator(rescale=1./255)

# Generators
train_generator = train_datagen.flow_from_directory(
    '/content/dogs_cats_dataset/dataset/training_set',   # updated path
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/dogs_cats_dataset/dataset/test_set',    # updated path
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)


In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.35),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    batch_size=16,
    callbacks=[early_stop]
)


In [ ]:
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")


In [ ]:
# Single Cell: Cat or Dog Prediction
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files

# Upload image
uploaded = files.upload()

for fn in uploaded.keys():
    img_path = fn
    print(f"Uploaded image: {img_path}")

    # Preprocess image
    img = image.load_img(img_path, target_size=(128, 128))  # same size as training
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # rescale
    img_array = np.expand_dims(img_array, axis=0)  # add batch dimension

    # Prediction
    prediction = model.predict(img_array)[0][0]  # output 0 to 1
    if prediction < 0.5:
        print("This is a CAT 🐱")
    else:
        print("This is a DOG 🐶")
